<a href="https://colab.research.google.com/github/vamshikrishnagolla/Api-routing/blob/main/StudentReport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json

class Student:
    _id_counter = 1

    def __init__(self, name):
        self.student_id = Student._id_counter
        Student._id_counter += 1
        self.name = name
        self.subjects = {}

    def add_subject(self, subject_name, marks):
        self.subjects[subject_name] = marks

    def calculate_average(self):
        if not self.subjects:
            return 0
        total = sum(self.subjects.values())
        return total / len(self.subjects)

    def get_grade(self):
        average = self.calculate_average()
        if average >= 90:
            return "A"
        elif average >= 75:
            return "B"
        elif average >= 50:
            return "C"
        else:
            return "Fail"

    def to_dict(self):
        return {
            "student_id": self.student_id,
            "name": self.name,
            "subjects": self.subjects
        }

    @staticmethod
    def from_dict(data):
        student = Student(data["name"])
        student.student_id = data["student_id"]
        student.subjects = data["subjects"]
        return student


In [ ]:
class GradeManager:
    def __init__(self):
        self.students = []

    def add_student(self, name, subjects):
        student = Student(name)
        for subject_name, marks in subjects.items():
            student.add_subject(subject_name, marks)
        self.students.append(student)
        print("Student added with ID:", student.student_id)

    def update_marks(self, student_id, subject_name, marks):
        student = self._find_student(student_id)
        if student:
            student.add_subject(subject_name, marks)
            print("Marks updated.")
        else:
            print("Student not found.")

    def view_report(self, student_id):
        student = self._find_student(student_id)
        if student:
            print(f"\n Report for {student.name} (ID: {student.student_id})")
            for subject_name, marks in student.subjects.items():
                print(f"{subject_name}: {marks}")
            print("Average:", student.calculate_average())
            print("Grade:", student.get_grade(), "\n")
        else:
            print("Student not found.")

    def delete_student(self, student_id):
        student = self._find_student(student_id)
        if student:
            self.students.remove(student)
            print("Student deleted.")
        else:
            print("Student not found.")

    def save_to_file(self, filename="grades.json"):
        data = [student.to_dict() for student in self.students]
        with open(filename, "w") as file:
            json.dump(data, file)
        print("Data saved successfully.")

    def load_from_file(self, filename="grades.json"):
        try:
            with open(filename, "r") as file:
                data = json.load(file)
                self.students = [Student.from_dict(item) for item in data]
                if self.students:
                    Student._id_counter = max(s.student_id for s in self.students) + 1
            print("Data loaded successfully.")
        except FileNotFoundError:
            print("No file found. Starting fresh.")

    def _find_student(self, student_id):
        for student in self.students:
            if student.student_id == student_id:
                return student
        return None


In [ ]:
def main():
    grade_manager = GradeManager()
    grade_manager.load_from_file()

    while True:
        print("\n======= Student Report Card Manager =======")
        print("1. Add Student")
        print("2. Update Marks")
        print("3. View Report")
        print("4. Delete Student")
        print("5. Save to File")
        print("6. Load from File")
        print("7. Exit")
        choice = input("Enter your choice (1–7): ")

        if choice == "1":
            name = input("Enter student name: ")
            subjects = {}
            while True:
                subject_name = input("Enter subject name (or 'done' to finish): ")
                if subject_name.lower() == "done":
                    break
                try:
                    marks = float(input(f"Enter marks for {subject_name}: "))
                    subjects[subject_name] = marks
                except ValueError:
                    print("Please enter valid marks.")
            grade_manager.add_student(name, subjects)

        elif choice == "2":
            try:
                student_id = int(input("Enter student ID: "))
                subject_name = input("Enter subject name: ")
                marks = float(input("Enter new marks: "))
                grade_manager.update_marks(student_id, subject_name, marks)
            except ValueError:
                print("Invalid input.")

        elif choice == "3":
            try:
                student_id = int(input("Enter student ID: "))
                grade_manager.view_report(student_id)
            except ValueError:
                print("Invalid input.")

        elif choice == "4":
            try:
                student_id = int(input("Enter student ID to delete: "))
                grade_manager.delete_student(student_id)
            except ValueError:
                print("Invalid input.")

        elif choice == "5":
            grade_manager.save_to_file()

        elif choice == "6":
            grade_manager.load_from_file()

        elif choice == "7":
            print("Exiting program. Goodbye!")
            break

        else:
            print("Invalid choice. Please select from 1 to 7.")


In [ ]:
if __name__ == "__main__":
    main()